# JS: Controls

> JavaScript generators for width, scale, and card count management.

In [ ]:
#| default_exp js.controls

In [ ]:
#| export
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.models import CardStackUrls
from cjm_fasthtml_card_stack.core.constants import (
    width_storage_key, scale_storage_key, card_count_storage_key,
    auto_count_storage_key,
    DEFAULT_CARD_WIDTH, DEFAULT_CARD_SCALE, DEFAULT_VISIBLE_COUNT,
)

## Width Management

Syncs the width slider value with localStorage and the card stack
inner container's `max-width`. Debounces server persistence.

In [ ]:
#| export
def _generate_width_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with slider bounds
    urls: CardStackUrls,  # URL bundle (save_width)
) -> str:  # JS code fragment for width management
    """Generate JS for width slider management."""
    storage_key = width_storage_key(config.prefix)
    return f"""
        // === Width Management ===
        const _WIDTH_KEY = '{storage_key}';
        let _saveWidthTimer = null;

        function _saveWidthToServer(val) {{
            if (!'{urls.save_width}') return;
            clearTimeout(_saveWidthTimer);
            _saveWidthTimer = setTimeout(function() {{
                htmx.ajax('POST', '{urls.save_width}', {{
                    swap: 'none', values: {{ card_width: val }}
                }});
            }}, 500);
        }}

        ns.updateWidth = function(value) {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            inner.style.maxWidth = value + 'rem';
            try {{ localStorage.setItem(_WIDTH_KEY, value); }} catch (e) {{}}
            const slider = document.getElementById('{ids.width_slider}');
            if (slider && parseInt(slider.value) !== parseInt(value)) slider.value = value;
            _saveWidthToServer(value);
            if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
        }};

        ns.decreaseWidth = function() {{
            const slider = document.getElementById('{ids.width_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_WIDTH};
            ns.updateWidth(Math.max({config.card_width_min}, current - {config.card_width_step}));
        }};

        ns.increaseWidth = function() {{
            const slider = document.getElementById('{ids.width_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_WIDTH};
            ns.updateWidth(Math.min({config.card_width_max}, current + {config.card_width_step}));
        }};

        ns.applyWidth = function() {{
            const inner = document.getElementById('{ids.card_stack_inner}');
            if (!inner) return;
            let val = {DEFAULT_CARD_WIDTH};
            try {{ const s = localStorage.getItem(_WIDTH_KEY); if (s) val = parseInt(s); }} catch (e) {{}}
            inner.style.maxWidth = val + 'rem';
            const slider = document.getElementById('{ids.width_slider}');
            if (slider) slider.value = val;
        }};
    """

In [ ]:
# Test width management JS generation
ids = CardStackHtmlIds(prefix="cs0")
config = CardStackConfig(prefix="cs0")
urls = CardStackUrls(save_width="/cs/save_width")

js = _generate_width_mgmt_js(ids, config, urls)
assert "Width Management" in js
assert ids.card_stack_inner in js
assert ids.width_slider in js
assert "ns.updateWidth" in js
assert "ns.decreaseWidth" in js
assert "ns.increaseWidth" in js
assert "ns.applyWidth" in js
assert "ns.triggerAutoAdjust" in js
assert urls.save_width in js
print("Width management JS tests passed!")

Width management JS tests passed!


## Scale Management

Syncs the scale slider value with localStorage and a CSS custom
property (`--card-stack-scale`). Debounces server persistence.

In [ ]:
#| export
def _generate_scale_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with slider bounds
    urls: CardStackUrls,  # URL bundle (save_scale)
) -> str:  # JS code fragment for scale management
    """Generate JS for scale slider management."""
    storage_key = scale_storage_key(config.prefix)
    return f"""
        // === Scale Management ===
        const _SCALE_KEY = '{storage_key}';
        let _saveScaleTimer = null;

        function _saveScaleToServer(val) {{
            if (!'{urls.save_scale}') return;
            clearTimeout(_saveScaleTimer);
            _saveScaleTimer = setTimeout(function() {{
                htmx.ajax('POST', '{urls.save_scale}', {{
                    swap: 'none', values: {{ card_scale: val }}
                }});
            }}, 500);
        }}

        function _applyScaleCssProperty(val) {{
            const cs = document.getElementById('{ids.card_stack}');
            if (cs) cs.style.setProperty('--card-stack-scale', val);
        }}

        ns.updateScale = function(value) {{
            _applyScaleCssProperty(value);
            try {{ localStorage.setItem(_SCALE_KEY, value); }} catch (e) {{}}
            const slider = document.getElementById('{ids.scale_slider}');
            if (slider && parseInt(slider.value) !== parseInt(value)) slider.value = value;
            _saveScaleToServer(value);
            if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
        }};

        ns.decreaseScale = function() {{
            const slider = document.getElementById('{ids.scale_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_SCALE};
            ns.updateScale(Math.max({config.card_scale_min}, current - {config.card_scale_step}));
        }};

        ns.increaseScale = function() {{
            const slider = document.getElementById('{ids.scale_slider}');
            const current = slider ? parseInt(slider.value) : {DEFAULT_CARD_SCALE};
            ns.updateScale(Math.min({config.card_scale_max}, current + {config.card_scale_step}));
        }};

        ns.applyScale = function() {{
            let val = {DEFAULT_CARD_SCALE};
            try {{ const s = localStorage.getItem(_SCALE_KEY); if (s) val = parseInt(s); }} catch (e) {{}}
            _applyScaleCssProperty(val);
            const slider = document.getElementById('{ids.scale_slider}');
            if (slider) slider.value = val;
        }};
    """

In [ ]:
# Test scale management JS generation
js = _generate_scale_mgmt_js(ids, config, urls)
assert "Scale Management" in js
assert ids.card_stack in js
assert ids.scale_slider in js
assert "ns.updateScale" in js
assert "ns.decreaseScale" in js
assert "ns.increaseScale" in js
assert "ns.applyScale" in js
assert "--card-stack-scale" in js
assert "ns.triggerAutoAdjust" in js
print("Scale management JS tests passed!")

Scale management JS tests passed!


## Card Count Management

Manages the visible card count dropdown, localStorage persistence,
and the auto/manual mode toggle. Fires HTMX viewport updates when
the count changes.

In [ ]:
#| export
def _generate_card_count_mgmt_js(
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    config: CardStackConfig,  # Config with count options
    urls: CardStackUrls,  # URL bundle (update_viewport)
) -> str:  # JS code fragment for card count management
    """Generate JS for card count selector management."""
    storage_key = card_count_storage_key(config.prefix)
    auto_key = auto_count_storage_key(config.prefix)
    valid_counts = ', '.join(str(c) for c in config.visible_count_options)
    return f"""
        // === Card Count Management ===
        const _COUNT_KEY = '{storage_key}';
        const _AUTO_KEY = '{auto_key}';
        const _VALID_COUNTS = [{valid_counts}];

        function _isAutoMode() {{
            try {{ return localStorage.getItem(_AUTO_KEY) === 'true'; }} catch (e) {{ return false; }}
        }}

        function _getStoredCount() {{
            try {{
                const s = localStorage.getItem(_COUNT_KEY);
                if (s) {{ const c = parseInt(s); if (_VALID_COUNTS.includes(c)) return c; }}
            }} catch (e) {{}}
            return {DEFAULT_VISIBLE_COUNT};
        }}

        ns.updateCardCount = function(value) {{
            const count = parseInt(value);
            if (!_VALID_COUNTS.includes(count)) return;
            try {{ localStorage.setItem(_COUNT_KEY, count); }} catch (e) {{}}
            const cardStack = document.getElementById('{ids.card_stack}');
            if (cardStack) cardStack.dataset.visibleCount = count;
            if ('{urls.update_viewport}') {{
                htmx.ajax('POST', '{urls.update_viewport}', {{
                    target: '#' + '{ids.card_stack}',
                    swap: 'none',
                    values: {{ visible_count: count }}
                }});
            }}
        }};

        ns._autoUpdateCount = function(count) {{
            // Like updateCardCount but bypasses _VALID_COUNTS validation.
            // Used by auto-adjustment which can set any count.
            const c = Math.max(1, Math.round(count));
            const cardStack = document.getElementById('{ids.card_stack}');
            if (cardStack) cardStack.dataset.visibleCount = c;
            if ('{urls.update_viewport}') {{
                htmx.ajax('POST', '{urls.update_viewport}', {{
                    target: '#' + '{ids.card_stack}',
                    swap: 'none',
                    values: {{ visible_count: c }}
                }});
            }}
        }};

        ns.handleCountChange = function(value) {{
            // Entry point for dropdown onchange — handles both \"auto\" and numeric values.
            if (value === 'auto') {{
                try {{ localStorage.setItem(_AUTO_KEY, 'true'); }} catch (e) {{}}
                if (ns.triggerAutoAdjust) ns.triggerAutoAdjust();
            }} else {{
                try {{ localStorage.removeItem(_AUTO_KEY); }} catch (e) {{}}
                if (ns._cancelAutoGrowth) ns._cancelAutoGrowth();
                ns.updateCardCount(parseInt(value));
            }}
        }};

        function _syncCountDropdown() {{
            const sel = document.getElementById('{ids.card_count_select}');
            if (!sel) return;
            if (_isAutoMode()) {{
                if (sel.value !== 'auto') sel.value = 'auto';
            }} else {{
                const stored = _getStoredCount();
                if (parseInt(sel.value) !== stored) sel.value = stored;
            }}
        }}

        // Expose for external callers (e.g., chrome swap after zone change)
        ns.syncCountDropdown = _syncCountDropdown;
    """

In [ ]:
# Test card count management JS generation
urls_full = CardStackUrls(update_viewport="/cs/update_viewport")
js = _generate_card_count_mgmt_js(ids, config, urls_full)
assert "Card Count Management" in js
assert ids.card_stack in js
assert ids.card_count_select in js
assert "ns.updateCardCount" in js
assert "ns._autoUpdateCount" in js
assert "ns.handleCountChange" in js
assert "ns.syncCountDropdown" in js  # Exposed for external callers
assert "_isAutoMode" in js
assert "_syncCountDropdown" in js
assert "_AUTO_KEY" in js
assert "swap: 'none'" in js
# Valid count options from config
assert "1, 3, 5, 7, 9" in js
print("Card count management JS tests passed!")

Card count management JS tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()